__This notebook is not the final used version__. 
The final used version can be found in the __vref_walkability_index repository.__
__Notebook 03_BT_01_Calculate_perception_variables.ipynb__

# 13 - Betas_v1

This notebook tries (and fails) to reproduce the calculation of the __first version__ of the betas (Value given from people in interviews to physical variables in the analysed territories) before notebook 03_BT_01_Calculate_perception_variables.ipynb was shared.

## Import libraries

In [1]:
first_folder_path = '../'

In [2]:
import warnings
warnings.filterwarnings('ignore')
import geopandas as gpd
from geopandas.tools import overlay
import matplotlib.pyplot as plt

import pandas as pd
import numpy as np
import seaborn as sns
import random
import scipy.stats as stats

import os
import sys
module_path = os.path.abspath(os.path.join(first_folder_path))
if module_path not in sys.path:
    sys.path.append(module_path)
    import src
else:
    import src

## Notebook config

## Input - Load surveys

In [3]:
# Read file (Converted from .xlsx to .csv UTF-8)
#'Encuesta_caminata(Guadalajara)(1-347).csv'
#'Encuesta_caminata(Guadalajara)(1-510).csv'
#'Encuesta_caminata(Guadalajara)(1-553).csv'

surveys_file = 'Encuesta_caminata(Guadalajara)(1-553).csv'
surveys = pd.read_csv(first_folder_path + f"data/output/table/encuestas/{surveys_file}")

# Show
print(surveys.shape)
surveys.head(2)

(553, 65)


,ID,Hora de inicio,Hora de finalización,Correo electrónico,Nombre,Hora de la última modificación,Encuestador,Fecha de realización,Encuesta #,Territorio de análisis,...,3,4,5,6,7,8,9,10,11,12
0,1,6/28/24 12:24:50,6/28/24 12:25:46,anonymous,NaN,NaN,DS,6/25/2024,001,TA_Miramar,...,"Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo"
1,2,6/28/24 12:25:58,6/28/24 12:28:18,anonymous,NaN,NaN,DS,6/25/2024,002,TA_Miramar,...,"Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo"


In [4]:
surveys.columns

Index(['ID', 'Hora de inicio', 'Hora de finalización', 'Correo electrónico',
       'Nombre', 'Hora de la última modificación', 'Encuestador',
       'Fecha de realización', 'Encuesta #', 'Territorio de análisis',
       'Lugar donde se realizó la encuesta', 'Género', 'Edad (años)',
       'Estrato socioeconómico (NO APLICA)', 'Ocupación principal',
       'Nivel educativo completado',
       '¿Poseen alguno de los siguientes vehículos en su vivienda?',
       'Indique su lugar de residencia (Municipio/comuna/barrio)',
       '¿Hace uso de algún equipamiento/sitio de interés localizado en el sector/territorio?',
       '¿Cómo suple las necesidades de equipamientos educativos?',
       '¿Cómo suple las necesidades de equipamientos de cultura o deportivos?',
       '¿Cuáles?',
       '¿Hacia qué tipo de equipamiento/sitio de interés se dirige?',
       '¿Qué medio de transporte utilizó para llegar al equipamiento X? (último viaje realizado)',
       'En una semana típica, ¿Cuántas veces 

## Filter for data of interest

In [5]:
ask_disability_question = True
ask_transport_question = True
ask_equipamiento_question = False
ask_veces_caminata_question = False

surveys_f = surveys.copy()

In [6]:
disability_question = '¿Se encuentra usted en alguna condición de discapacidad?'

if ask_disability_question:
    print(surveys[disability_question].unique())
    disability_lst = ['Si']
    surveys_f = surveys_f.loc[~surveys_f[disability_question].isin(disability_lst)].copy()

    # Show
    print(surveys_f.shape)
    surveys_f.head(2)

[nan 'No' 'Si']
(547, 65)


In [7]:
transport_question = '¿Qué medio de transporte utilizó para llegar al equipamiento X? (último viaje realizado)'

if ask_transport_question:
    print(surveys[transport_question].unique())
    modes_lst = ['Caminata']
    surveys_f = surveys_f.loc[surveys_f[transport_question].isin(modes_lst)].copy()

    # Show
    print(surveys_f.shape)
    surveys_f.head(2)

['Uber' 'Carro particular - coche' 'Caminata' 'Mototaxi'
 'SITVA (Metro, Metrocable, Metroplus, Tranvía, Bus alimentador)'
 'Motocicleta' 'Transporte Centro de Rehabilitación (Arenales)' 'Bus'
 'Plataformas digitales (Uber, Didi, InDriver, Picap, etc.)' 'Taxi'
 'Transporte privado de parte de la empresa donde labora' 'Bicicleta'
 'Transporte empresarial' 'Patineta eléctrica' 'Ride' 'Transporte privado']
(209, 65)


In [8]:
equipamiento_question = '¿Hace uso de algún equipamiento/sitio de interés localizado en el sector/territorio?'

if ask_equipamiento_question:
    print(surveys[equipamiento_question].unique())
    equip_lst = ['Si']
    surveys_f = surveys_f.loc[surveys_f[equipamiento_question].isin(equip_lst)].copy()

    # Show
    print(surveys_f.shape)
    surveys_f.head(2)

In [9]:
veces_caminata_question = 'En una semana típica, ¿Cuántas veces realiza este mismo viaje utilizando la caminata como modo de transporte? (CAMINATA)'

if ask_veces_caminata_question:
    print(surveys[veces_caminata_question].unique())
    surveys_f = surveys_f.loc[~surveys_f[veces_caminata_question].isna()].copy()

    # Show
    print(surveys_f.shape)
    surveys_f.head(2)

In [10]:
print(f"Colinas {len(surveys_f.loc[surveys_f['Territorio de análisis']=='TA_Colinas'])}.")
print(f"Miramar {len(surveys_f.loc[surveys_f['Territorio de análisis']=='TA_Miramar'])}.")
print(f"Providencia {len(surveys_f.loc[surveys_f['Territorio de análisis']=='TA_Providencia'])}.")

Colinas 13.
Miramar 124.
Providencia 72.


## Rename columns

In [11]:
# INFO - Columns that hold relation with physical variables. According to the paper:

#The construction of the questions asked to the walkers, seek to capture the perception regarding
#the territories they travel on their journeys, so that the contribution of their answers is a positive
#beta if they have a good perception and so that it is negative if they have a bad perception;
#However, some questions, particularly regarding the slope and the length of the route, contributed
#in the opposite sense, which is why it was necessary to make a correction: each of the answers is
#multiplied by (-1) so that their answer contributes in the correct sense, and in the same way the
#interpretation of the question is corrected.

# Therefore:

# POPULATION DENSITY
# 'Column' --> 36. (A lo largo de su recorrido, ¿Sientes que hay personas a tu alrededor que pudieran apoyarte en caso de algún problema?)
# Sí --> GOOD PERCEPTION (Hay densidad)
# No --> BAD PERCEPTION (No hay densidad)

# INTERSECTIONS DENSITY
# '2' --> 37. (A lo largo de su recorrido, ¿Sientes que hay diversas intersecciones que te permiten cruzar las calles de manera segura?)
# Sí --> GOOD PERCEPTION (Hay densidad de intersecciones)
# No --> BAD PERCEPTION (Faltan intersecciones)

# '3' --> 38. (¿Se te hace largo y aburrido el recorrido?) [INVERSE]
# Sí --> BAD PERCEPTION (Faltan intersecciones para diversificar el recorrido) [INVERSE]
# No --> GOOD PERCEPTION (Hay intersecciones que diversifican el recorrido) [INVERSE]

# '4' --> 39. (A lo largo de tu recorrido, ¿Es posible desviarse en múltiples puntos/intersecciones para cambiar de ruta?)
# Sí --> GOOD PERCEPTION (Hay densidad de intersecciones)
# No --> BAD PERCEPTION (Faltan intersecciones)

# LAND USE
# '5' --> 40. (A lo largo de tu recorrido, ¿Puedes aprovechar para acceder a múltiples servicios y satisfacer necesidades diversas (realizar vueltas/pendientes, comprar alimentos, divertirse, etc.)?)
# Sí --> GOOD PERCEPTION (Hay varios usos)
# No --> BAD PERCEPTION (Faltan usos)

# '6' --> 41. (A lo largo de tu recorrido, ¿Sientes que la comodidad en tu caminar se ve influenciada por la cantidad de personas y actividades que se realizan en el andén?)
# Sí --> GOOD PERCEPTION (Hay varias personas realizando actividades)
# No --> BAD PERCEPTION (La cantidad de personas/actividades no influencía la comodidad al caminar)

# '7' --> 42. (A lo largo de tu recorrido, ¿Encuentras elementos que hacen que tu caminata sea poco monótona?)
# Sí --> GOOD PERCEPTION (Hay varios usos)
# No --> BAD PERCEPTION (Faltan usos)

# VEGETATION INDEX
# '8' --> 43. (A lo largo de tu recorrido, ¿Sientes que el tipo de vegetación te genera sombra y protección del sol?)
# Sí --> GOOD PERCEPTION (Hay vegetación)
# No --> BAD PERCEPTION (Falta vegetación)

# '9' --> 44. (A lo largo de tu recorrido, ¿Sientes que la presencia y el tipo de vegetación crea un entorno agradable?)
# Sí --> GOOD PERCEPTION (Hay vegetación)
# No --> BAD PERCEPTION (Falta vegetación)

# SLOPE
# '10' --> 45. (A lo largo de tu recorrido, ¿Consideras que el hecho de caminar te implicó un gran esfuerzo físico debido a la pendiente/inclinación?) [INVERSE]
# Sí --> BAD PERCEPTION (Hay pendiente) [INVERSE]
# No --> GOOD PERCEPTION (No hay pendiente) [INVERSE]

# SIDEWALK PRESENCE
# '11' --> 46. (A lo largo de tu recorrido, ¿Te sientes seguro(a) ante la presencia de vehículos?)
# Sí --> GOOD PERCEPTION (La infraestructura te hace sentir seguro)
# No --> BAD PERCEPTION (La infraestructura no es suficiente para hacerte sentir seguro)

# '11' --> 47. (A lo largo de tu recorrido, ¿Sientes que la continuidad de andenes te permite caminar de manera cómoda (no hay: cambios de niveles, interrupciones repentinas del andén, presencia de obstáculos, cambios en el ancho del andén, etc.)?)
# Sí --> GOOD PERCEPTION (La infraestructura te permite andar cómodamente)
# No --> BAD PERCEPTION (La infraestructura no es suficiente para permitirte andar cómodamente)

In [12]:
# Tested different survey files, column name unification:

if surveys_file == 'Encuesta_caminata(Guadalajara)(1-553).csv':
    # Filter for data of interest
    surveys_f = surveys_f[['ID','Territorio de análisis','Column', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12']]
    
    # Rename columns
    surveys_f.rename(columns={'Territorio de análisis':'territory',
                              'Column':'36_text',
                              '2':'37_text',
                              '3':'38_text',
                              '4':'39_text',
                              '5':'40_text',
                              '6':'41_text',
                              '7':'42_text',
                              '8':'43_text',
                              '9':'44_text',
                              '10':'45_text',
                              '11':'46_text',
                              '12':'47_text'
                             },inplace=True
                    )
else:
    # Filter for data of interest
    surveys_f = surveys_f[['ID','Territorio de análisis',
                         '36. A lo largo de su recorrido, ¿Sientes que hay personas a tu alrededor que pudieran apoyarte en caso de algún problema?',
                         '37. A lo largo de su recorrido, ¿Sientes que hay diversas intersecciones que te permiten cruzar las calles de manera segura?',
                         '38. ¿Se te hace largo y aburrido el recorrido?',
                         '39. A lo largo de tu recorrido, ¿Es posible desviarse en múltiples puntos/intersecciones para cambiar de ruta?',
                         '40. A lo largo de tu recorrido, ¿Puedes aprovechar para acceder a múltiples servicios y satisfacer necesidades diversas (realizar vueltas/pendientes, comprar alimentos, divertirse, etc.)?',
                         '41. A lo largo de tu recorrido, ¿Sientes que la comodidad en tu caminar se ve influenciada por la cantidad de personas y actividades que se realizan en el andén?',
                         '42. A lo largo de tu recorrido, ¿Encuentras elementos que hacen que tu caminata sea poco monótona?',
                         '43. A lo largo de tu recorrido, ¿Sientes que el tipo de vegetación te genera sombra y protección del sol?',
                         '44. A lo largo de tu recorrido, ¿Sientes que la presencia y el tipo de vegetación crea un entorno agradable?',
                         '45. A lo largo de tu recorrido, ¿Consideras que el hecho de caminar te implicó un gran esfuerzo físico debido a la pendiente/inclinación?',
                         '46. A lo largo de tu recorrido, ¿Te sientes seguro(a) ante la presencia de vehículos?',
                         '47. A lo largo de tu recorrido, ¿Sientes que la continuidad de andenes te permite caminar de manera cómoda (no hay: cambios de niveles, interrupciones repentinas del andén, presencia de obstáculos, cambios en el ancho del andén, etc.)?'
                        ]]
    # Rename columns
    surveys_f.rename(columns={'Territorio de análisis':'territory',
                              '36. A lo largo de su recorrido, ¿Sientes que hay personas a tu alrededor que pudieran apoyarte en caso de algún problema?':'36_text',
                              '37. A lo largo de su recorrido, ¿Sientes que hay diversas intersecciones que te permiten cruzar las calles de manera segura?':'37_text',
                              '38. ¿Se te hace largo y aburrido el recorrido?':'38_text',
                              '39. A lo largo de tu recorrido, ¿Es posible desviarse en múltiples puntos/intersecciones para cambiar de ruta?':'39_text',
                              '40. A lo largo de tu recorrido, ¿Puedes aprovechar para acceder a múltiples servicios y satisfacer necesidades diversas (realizar vueltas/pendientes, comprar alimentos, divertirse, etc.)?':'40_text',
                              '41. A lo largo de tu recorrido, ¿Sientes que la comodidad en tu caminar se ve influenciada por la cantidad de personas y actividades que se realizan en el andén?':'41_text',
                              '42. A lo largo de tu recorrido, ¿Encuentras elementos que hacen que tu caminata sea poco monótona?':'42_text',
                              '43. A lo largo de tu recorrido, ¿Sientes que el tipo de vegetación te genera sombra y protección del sol?':'43_text',
                              '44. A lo largo de tu recorrido, ¿Sientes que la presencia y el tipo de vegetación crea un entorno agradable?':'44_text',
                              '45. A lo largo de tu recorrido, ¿Consideras que el hecho de caminar te implicó un gran esfuerzo físico debido a la pendiente/inclinación?':'45_text',
                              '46. A lo largo de tu recorrido, ¿Te sientes seguro(a) ante la presencia de vehículos?':'46_text',
                              '47. A lo largo de tu recorrido, ¿Sientes que la continuidad de andenes te permite caminar de manera cómoda (no hay: cambios de niveles, interrupciones repentinas del andén, presencia de obstáculos, cambios en el ancho del andén, etc.)?':'47_text'
                             },inplace=True
                    )

# Show
surveys_f.head(2)

,ID,territory,36_text,37_text,38_text,39_text,40_text,41_text,42_text,43_text,44_text,45_text,46_text,47_text
3,4,TA_Miramar,Totalmente de acuerdo,Totalmente en desacuerdo,Totalmente en desacuerdo,Totalmente de acuerdo,Totalmente de acuerdo,Totalmente de acuerdo,Totalmente en desacuerdo,Totalmente en desacuerdo,Totalmente en desacuerdo,Totalmente de acuerdo,Totalmente en desacuerdo,Totalmente en desacuerdo
7,8,TA_Miramar,Totalmente de acuerdo,Totalmente en desacuerdo,Totalmente en desacuerdo,Totalmente de acuerdo,Totalmente de acuerdo,Totalmente de acuerdo,Totalmente en desacuerdo,Totalmente en desacuerdo,Totalmente en desacuerdo,Totalmente de acuerdo,Totalmente en desacuerdo,Totalmente en desacuerdo


## Assign values to survey's answers

In [13]:
# Mapping dictionary (Assigns value to answer)
mapping_dct = {"Totalmente en desacuerdo": -2,
               "En desacuerdo": -1,
               "Ni de acuerdo, ni en desacuerdo": 0,
               "De acuerdo": 1,
               "Totalmente de acuerdo": 2
}

# Mapping dictionary for questions 
#38. (¿Se te hace largo y aburrido el recorrido?) and
#45. (A lo largo de tu recorrido, ¿Consideras que el hecho de caminar te implicó un gran esfuerzo físico debido a la pendiente/inclinación?)
# where answers pivoting towards "yes" indicate a bad perception of the physical variable (lack of intersecions, presence of slope)
# and answers pivoting towards "no" indicate good perception of the physical variable (intersecions density, no slope)
inverse_mapping_dct = {"Totalmente en desacuerdo": 2,
                       "En desacuerdo": 1,
                       "Ni de acuerdo, ni en desacuerdo": 0,
                       "De acuerdo": -1,
                       "Totalmente de acuerdo": -2
}

# Apply mappings
question_numbers = ['36','37','38','39','40','41','42','43','44','45','46','47']
for question in question_numbers:
    # Apply regular mapping to all except question 38 and 45
    if (question != '38') and (question!='45'):
        surveys_f[question+'_value'] = surveys_f[question+'_text'].map(mapping_dct)
    # Apply inverse mapping to question 38
    elif question in ['38', '45']:
        surveys_f[question+'_value'] = surveys_f[question+'_text'].map(inverse_mapping_dct)

# Reorder columns to visualize
ordered_cols = []
for question in question_numbers:
    ordered_cols.append(question+'_text')
    ordered_cols.append(question+'_value')
surveys_f = surveys_f[['ID','territory']+ordered_cols]

# Show
surveys_f.head(5)

,ID,territory,36_text,36_value,37_text,37_value,38_text,38_value,39_text,39_value,...,43_text,43_value,44_text,44_value,45_text,45_value,46_text,46_value,47_text,47_value
3,4,TA_Miramar,Totalmente de acuerdo,2,Totalmente en desacuerdo,-2,Totalmente en desacuerdo,2,Totalmente de acuerdo,2,...,Totalmente en desacuerdo,-2,Totalmente en desacuerdo,-2,Totalmente de acuerdo,-2,Totalmente en desacuerdo,-2,Totalmente en desacuerdo,-2
7,8,TA_Miramar,Totalmente de acuerdo,2,Totalmente en desacuerdo,-2,Totalmente en desacuerdo,2,Totalmente de acuerdo,2,...,Totalmente en desacuerdo,-2,Totalmente en desacuerdo,-2,Totalmente de acuerdo,-2,Totalmente en desacuerdo,-2,Totalmente en desacuerdo,-2
9,10,TA_Miramar,Totalmente de acuerdo,2,Totalmente en desacuerdo,-2,Totalmente en desacuerdo,2,Totalmente de acuerdo,2,...,Totalmente en desacuerdo,-2,Totalmente en desacuerdo,-2,Totalmente en desacuerdo,2,Totalmente en desacuerdo,-2,Totalmente en desacuerdo,-2
10,11,TA_Miramar,Totalmente de acuerdo,2,Totalmente de acuerdo,2,Totalmente en desacuerdo,2,Totalmente en desacuerdo,-2,...,Totalmente en desacuerdo,-2,Totalmente de acuerdo,2,Totalmente en desacuerdo,2,Totalmente de acuerdo,2,Totalmente en desacuerdo,-2
11,12,TA_Miramar,Totalmente de acuerdo,2,Totalmente en desacuerdo,-2,Totalmente en desacuerdo,2,Totalmente de acuerdo,2,...,"Ni de acuerdo, ni en desacuerdo",0,Totalmente en desacuerdo,-2,Totalmente en desacuerdo,2,Totalmente en desacuerdo,-2,Totalmente en desacuerdo,-2


In [14]:
for col in ordered_cols:
    test = surveys_f.loc[surveys_f[col].isna()]
    if len(test)>0:
        print(f"Nans in {col}.")
    else:
        print(f"No nans in col {col}.")

No nans in col 36_text.
No nans in col 36_value.
No nans in col 37_text.
No nans in col 37_value.
No nans in col 38_text.
No nans in col 38_value.
No nans in col 39_text.
No nans in col 39_value.
No nans in col 40_text.
No nans in col 40_value.
No nans in col 41_text.
No nans in col 41_value.
No nans in col 42_text.
No nans in col 42_value.
No nans in col 43_text.
No nans in col 43_value.
No nans in col 44_text.
No nans in col 44_value.
No nans in col 45_text.
No nans in col 45_value.
No nans in col 46_text.
No nans in col 46_value.
No nans in col 47_text.
No nans in col 47_value.


## Calculate average by question, by territory

In [15]:
question_summary_df = pd.DataFrame()

for territory in list(surveys_f.territory.unique()):

    # Separate by territory
    surveys_f_territory = surveys_f.loc[surveys_f.territory==territory].copy()
    
    rows = []
    for question in question_numbers:
        # Calculate average for current question in current territory
        question_avg = surveys_f_territory[question+'_value'].mean()
    
        # Register average for current question in current territory
        rows.append({'question': question, territory: question_avg})

    # Register question data in question_summary_df
    territory_df = pd.DataFrame(rows)
    if question_summary_df.empty:
        question_summary_df = territory_df
    else:
        question_summary_df = pd.merge(question_summary_df, territory_df, on='question')
    
question_summary_df

,question,TA_Miramar,TA_Providencia,TA_Colinas
0,36,0.645161,0.597222,0.692308
1,37,-0.201613,0.902778,0.000000
2,38,1.233871,1.430556,1.769231
3,39,0.741935,1.069444,0.538462
4,40,1.153226,1.013889,0.615385
5,41,0.483871,0.486111,-0.230769
6,42,-0.225806,0.194444,0.230769
7,43,-0.395161,1.236111,0.692308
8,44,-0.096774,1.458333,0.846154
9,45,0.814516,1.569444,1.153846


## Calculate betas

In [16]:
betas_df = pd.DataFrame()

questions_variables_dct = {'population_dens':['36'],
                           'intersections_dens':['37','38','39'],
                           'land_use':['40','41','42'],
                           'vegetation':['43','44'],
                           'slope':['45'],
                           'sidewalk':['46','47']
                          }

# Calculate betas by variable for each territory
for territory in list(surveys_f.territory.unique()):
    
    # Each territory's data is registered as a dict. in rows
    rows = []
    
    for variable, question_list in questions_variables_dct.items():
        
        # List all question average values
        var_values = []
        for question in question_list:
            idx_question = question_summary_df.question==question
            question_avg = question_summary_df.loc[idx_question,territory].values[0]
            var_values.append(question_avg)

        # Get an average of those averages
        variable_avg = np.mean(var_values)

        # Register in rows
        rows.append({'variable': variable, territory: variable_avg})
        
    # Register territory data in betas_df
    territory_df = pd.DataFrame(rows)
    if betas_df.empty:
        betas_df = territory_df
    else:
        betas_df = pd.merge(betas_df, territory_df, on='variable')

betas_df

,variable,TA_Miramar,TA_Providencia,TA_Colinas
0,population_dens,0.645161,0.597222,0.692308
1,intersections_dens,0.591398,1.134259,0.769231
2,land_use,0.470430,0.564815,0.205128
3,vegetation,-0.245968,1.347222,0.769231
4,slope,0.814516,1.569444,1.153846
5,sidewalk,-0.552419,0.513889,-0.192308
